In [7]:
import json
import multiprocessing
import os
import re
import tempfile
from collections import Counter
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import plyvel
import polars as pl
import whoosh
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from whoosh.analysis import StandardAnalyzer

In [2]:
df = pl.read_parquet("/kaggle/input/uspto-boolean-search-optimization/patent_metadata.parquet")
df.head(1)

publication_number,publication_date,filing_date,family_id,cpc_codes
str,datetime[μs],datetime[μs],f64,list[str]
"""US-1-A""",1836-07-13 00:00:00,null,2.060279e6,"[""B61C11/04""]"


In [8]:
# データベースの作成
db = plyvel.DB("db", create_if_missing=True)
for patent, cpcs in tqdm(zip(df["publication_number"], df["cpc_codes"].to_list())):
    db.put(patent.encode(), json.dumps(cpcs).encode())

13307751it [01:09, 191659.13it/s]


In [13]:
# 速度を確認
for patent in tqdm(df["publication_number"]):
    cpcs = json.loads(db.get(patent.encode()).decode())
cpcs

100%|██████████| 13307751/13307751 [00:32<00:00, 409878.07it/s]


[]

In [14]:
db.close()